# Counting, proofreading and visualizing inputs and outputs

This notebook requires an environment where the python <a href="https://navis.readthedocs.io/en/latest/" target="_blank">navis</a> and <a href="https://fafbseg-py.readthedocs.io/en/latest/source/api.html" target="_blank">fafbseq</a> packages are installed, apart from other well-known packages listed in the import below.
<br>**CONTAINS:**
<br>A) Analysis of inputs and ouputs based on *cleft score* value
<br>B) Analysis of inputs and outputs of a list of neurons
<br>C) Proofreading predicted synapses via annotations
<br>D) Calculating and plotting synaptic cleft size ditribution between partners 
<br>E) Checking the existance of IDs from Codex in our data base 
<br>F) Updating your IDs
<br>G) Plotting our Tm9 patches / clusters
<br>H) Tm9-patch mining

#### Importing packages

In [70]:
#Importing packages
import navis
import fafbseg
from fafbseg import flywire
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from caveclient import CAVEclient
from visualizations.helpers.synapse_queries import combine_xyz, separate_xyz, synapse_count

client = CAVEclient('flywire_fafb_production')

#### Setting a "secret" for accessing data

To get your personal secret, please go to: <a href="https://fafbseg-py.readthedocs.io/en/latest/source/tutorials/flywire_secret.html" target="_blank">get my secret.</a>

In [ ]:
# fafbseg.flywire.set_chunkedgraph_secret("5719b2db462d94d6aa0e903c1ff889e4") # 2022.03.11
# fafbseg.flywire.set_chunkedgraph_secret("c161f679dd3d52b1d5fc19f62cdd0164") # 2022.04.11 
fafbseg.flywire.set_chunkedgraph_secret("0c0edb71c5682a971dbadc37bcbabf29") # 2022.06.21 

## A) Analysis of inputs and ouputs based on *cleft score* value

To automatically detect of pre- and postsynaptic sites, the algorithm used in <a href="https://www.nature.com/articles/s41592-021-01183-7" target="_blank">Buhman et al. 2021.</a>
<br>A visualization of all predicted synapses can visualized by Neuroglancer <a href="http://www.tinyurl.com/tdq6xkw" target="_blank">here.</a>
<br>The algorithm returns a list of connections with two scores: the cleft score and the connection score. Number of detected synapses depend on them.

<font size="4"> The *cleft score* matters, the *connection score* not that much:</font> <br/>
<span style="color:blue">&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; True positives </span> <br/>
<span style="color:Chocolate">&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; False positives </span> <br/>
<span style="color:DimGrey">&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Unkown </span> <br/>


<img src="./Images/proofreading/szi-chieh_labels_synapse_eval_cleft_score.png" 
     alt="cleft_score"
     width="300" height="300" style="float: left ; width: 40%; padding-left: 10%"/>
<img src="./Images/proofreading/szi-chieh_labels_synapse_eval_synful_score.png" 
     alt="connection_score"
     width="300" height="300" style="float: left ; width: 40%; padding-left: 0%"/>



### Testing different cleft score values in Tm9 inputs

In the following cells, we choose different *cleft score* values to get the total number of inputs and outputs of a single Tm9.

In [ ]:
#Getting inputs and outputs for one neuron with different min_score values

# FIRST, Check if root IDs are outdated (i.e. have more recent edits)
r1 = 720575940626482442 # root ID query for Tm9
print(f'ID up-to-date in FlyWire: {flywire.is_latest_root([r1])}')
# SECOND, proceed getting the neurons data if the ID is up to date
if flywire.is_latest_root([r1])[0]:
    neuron = flywire.get_mesh_neuron(r1) # mesh query
else:
    print(f'ID not up to date. Update it and run again')

# THIRD, proceed getting the neuron's inputs and outputs for a list of different min_scores 
# “Cleft score” for automatic synaptic detection, Buhmann et al., 2021
if flywire.is_latest_root([r1])[0]:
    min_score_ls = [0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180]
    in_count_ls =[]
    out_count_ls =[]
    pre_count_ls=[]
    post_count_ls=[]
    
    for score in min_score_ls:
        # Fetch the neuron's inputs and ouputs
        inputs = flywire.synapses.fetch_synapses(neuron, pre=False, post=True, attach=True, 
                                                 min_score=score, clean=True, transmitters=False, 
                                                 neuropils=True, live_query=True, batch_size=30, 
                                                 dataset='production', progress=True)
        outputs = flywire.synapses.fetch_synapses(neuron, pre=True, post=False, attach=True, 
                                                 min_score=score, clean=True, transmitters=False, 
                                                 neuropils=True, live_query=True, batch_size=30, 
                                                 dataset='production', progress=True)
        
        # Counting inputs and ouputs per ID
        inputs_count = {}
        inputs_str = inputs.applymap(str)
        for c in inputs_str['pre'].to_list():
            inputs_count[c] = inputs_count.get(c, 0) + 1
        input_count_df = pd.DataFrame(inputs_count, index=[0])
        input_count_df = input_count_df.T
        input_count_df.rename(columns={0: "counts"},inplace=True)
        input_count_df.index.names = ['presynaptic_ID']
        input_count_df['postsynaptic_ID'] = inputs_str['post'].to_list()[0:len(input_count_df)]
        

        outputs_count = {}
        outputs_str = outputs.applymap(str)
        for c in outputs_str['post'].to_list():
            outputs_count[c] = outputs_count.get(c, 0) + 1
        output_count_df = pd.DataFrame(outputs_count, index=[0])
        output_count_df = output_count_df.T
        output_count_df.rename(columns={0: "counts"},inplace=True)
        output_count_df.index.names = ['postsynaptic_ID']
        output_count_df['presynaptic_ID'] = outputs_str['pre'].to_list()[0:len(output_count_df)]
        
        #Storing useful info during the for-loop
        in_count_ls.append(input_count_df['counts'].sum())
        out_count_ls.append(output_count_df['counts'].sum())
        pre_count_ls.append(len(input_count_df.index))
        post_count_ls.append(len(output_count_df.index))
        
        # Printing useful info
        #print(f"Current score = {score}")
        #print(f"Total number of inputs = {input_count_df['counts'].sum()}")
        #print(f"Total number of outputs = {output_count_df['counts'].sum()}")
        #print(f"Total number of presynaptic partners = {len(input_count_df.index)}")
        #print(f"Total number of postsynaptic partners = {len(output_count_df.index)}")

Let's now creat a pandas dataframe with the obtain lists and then generate some line plots.

In [ ]:
#Generation of pandas dataframe from lists
cleft_score_df = pd.DataFrame(list(zip(min_score_ls, in_count_ls, out_count_ls, pre_count_ls, post_count_ls)),
               columns =['Cleft_scores', 'input_count', 'output_count','pre_partner_count', 'post_partner_count'])
#print(cleft_score_df)

In [ ]:
#Plotting
fig, ax1 = plt.subplots()
color = 'tab:blue'
ax1.plot(cleft_score_df['Cleft_scores'],cleft_score_df['input_count'], color=color)
    ax1.set_ylabel('inputs', color=color) 
    ax1.set_title('Inputs and outputs across cleft scores')
    ax1.set_xlabel('minimun cleft score')
    ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:orange'
ax2.set_ylabel('outputs', color=color)  
ax2.plot(cleft_score_df['Cleft_scores'],cleft_score_df['output_count'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# Saving excel files
# Setting variable
outDir = r'Z:\Connectomics data\FlyWire\Excels\min_score_50'
save_excel_file = False

if save_excel_file: 
    ## Inputs
    file_name = str(neuron.id)+'_inputs.xlsx'
    savePath = os.path.join(outDir, file_name)
    inputs_str = inputs.applymap(str)
    inputs_str.to_excel(savePath)

    file_name = str(neuron.id)+'_inputs_count.xlsx'
    savePath = os.path.join(outDir, file_name)
    input_count_str_df = input_count_df.applymap(str)
    input_count_str_df.to_excel(savePath)


    ## Outputs
    file_name = str(neuron.id)+'_outputs.xlsx'
    savePath = os.path.join(outDir, file_name)
    outputs_str = outputs.applymap(str)
    outputs_str.to_excel(savePath)

    file_name = str(neuron.id)+'_outputs_count.xlsx'
    savePath = os.path.join(outDir, file_name)
    output_count_str_df = output_count_df.applymap(str)
    output_count_str_df.to_excel(savePath)

## B) Analysis of inputs and outputs of a list of neurons

### 1. Loading IDs manually

In [ ]:
# Check if root IDs are outdated (i.e. have more recent edits)
# Selecting Neurons
curr_ID_ls = [720575940623883277
,720575940610269508
,720575940629782335
,720575940629271370]   # Remaining Tm1 to proofread for sparse selection of L optic lobe

print(f'ID up-to-date in FlyWire: {flywire.is_latest_root(curr_ID_ls)}')

#### 1.1 Cheking connectivity of not-up-to-date segments

In [ ]:
if not np.unique(flywire.is_latest_root(curr_ID_ls))[0]: #if not up-to-date
    # Fetch the neuron's inputs and outputs
    neurons_inputs = flywire.synapses.fetch_synapses(curr_ID_ls, pre=False, post=True, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True) #,mat= "live"
    neurons_outputs = flywire.synapses.fetch_synapses(curr_ID_ls, pre=True, post=False, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True) #, mat= "live"


### 1. Or, loading from excel file

In [ ]:
# Choose path and file
import os

dataPath = r'D:\Connectomics data\FlyWire\Excels'
dataPath = r'C:\Users\sebas\Downloads'
fileDate = '20230110'
fileName = f'Tm9 proofreadings_{fileDate}.xlsx'
fileName = f'Tm1 proofreadings_{fileDate}.xlsx'
filePath = os.path.join(dataPath,fileName)

In [ ]:
df = pd.read_excel(filePath)
curr_ID_ls = df["seg_id"].tolist()

In [ ]:
# Applying any filter?
filtered_df = df[(df['twigs proofread (Y/N)']== 'Y') & (df['inputs_proofread (Y/N)']!= 'Y') & (df['cluster_id']!= -1)& (df['cluster_id']!= 3)].copy()
print(len(filtered_df))

# Defining id to work with
curr_ID_ls = filtered_df["seg_id"].tolist()

### 1. Or, loading from txt file

In [ ]:
# Choose path and file
import os

dataPath = r'E:\Connectomics-Data\FlyWire\Txts\cell_type_proofread'
fileDate = '20230614'
fileName = f'root_ids_Tm16_side_right.txt'
filePath = os.path.join(dataPath,fileName)
ids_df = pd.read_csv(filePath, sep = ",")
curr_ID_ls = ids_df.columns.tolist()

### 2. Updating IDs and fetching synapses

In [ ]:
#Updating the IDs via Fafbseg
updated_ID_df = fafbseg.flywire.update_ids(curr_ID_ls, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)

In [ ]:
# Fetching synapses from new IDs
ID_ls = updated_ID_df["new_id"].tolist()

if np.unique(flywire.is_latest_root(ID_ls))[0]:
    # Fetch the neuron's inputs and putputs
    neurons_inputs = flywire.synapses.fetch_synapses(ID_ls, pre=False, post=True, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")
    neurons_outputs = flywire.synapses.fetch_synapses(ID_ls, pre=True, post=False, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True, mat= "live")
    #synaptic_counts = flywire.synapses.synapse_counts(root_ids, by_neuropil=False, min_score=30, live_query=True,batch_size=10, dataset='production')
else:
    print(f'IDs not up to date, analysis aborted') 
    

#### Selecting a neuropile of interest for synaptic counts per neuron ID

In [ ]:
neurons_inputs.head()

In [ ]:
neurons_outputs.head()

In [ ]:
# Selecting inputs from a single neuropile
neuropile = ['ME_R','ME_R'] # String. 'LO_R', 'ME_R', ...
#or
neuropile_list = ['ME_R','LO_R']

neuropile_neurons_inputs = neurons_inputs[(neurons_inputs['neuropil'].isin(neuropile_list))]
no_neuropile_neurons_inputs = neurons_inputs[~(neurons_inputs['neuropil'].isin(neuropile))]


In [ ]:
no_neuropile_neurons_inputs

### 2.1. For all inputs

In [ ]:
# LOOPING. Counting inputs and ouputs per ID, option joining dataframes
final_input_df = pd.DataFrame()
for n in neurons_inputs['post'].unique():
    inputs_count = {}
    curr_inputs = neurons_inputs[neurons_inputs['post'] == n]
    inputs_str = curr_inputs.applymap(str)
    
    for c in inputs_str['pre'].to_list():
        inputs_count[c] = inputs_count.get(c, 0) + 1
    input_count_df = pd.DataFrame(inputs_count, index=[0])
    input_count_df = input_count_df.T
    input_count_df.rename(columns={0: "counts"},inplace=True)
    input_count_df.index.names = ['presynaptic_ID']
    input_count_df = input_count_df.sort_values(by="counts",ascending=False)
    input_count_df['postsynaptic_ID'] = inputs_str['post'].to_list()[0:len(input_count_df)]
    final_input_df = final_input_df.append(input_count_df)
    #print(f'Counting done for: {n}')
input_count_str_df = final_input_df.applymap(str)
input_count_str_df.head()

### 2.2 Or, for a selection of all inputs.

In [ ]:
# LOOPING. Counting inputs and ouputs per ID, option joining dataframes
#For "no_neuropile_neurons_inputs"

final_input_df = pd.DataFrame()
for n in no_neuropile_neurons_inputs['post'].unique():
    inputs_count = {}
    curr_inputs = no_neuropile_neurons_inputs[no_neuropile_neurons_inputs['post'] == n]
    inputs_str = curr_inputs.applymap(str)
    
    for c in inputs_str['pre'].to_list():
        inputs_count[c] = inputs_count.get(c, 0) + 1
    input_count_df = pd.DataFrame(inputs_count, index=[0])
    input_count_df = input_count_df.T
    input_count_df.rename(columns={0: "counts"},inplace=True)
    input_count_df.index.names = ['presynaptic_ID']
    input_count_df = input_count_df.sort_values(by="counts",ascending=False)
    input_count_df['postsynaptic_ID'] = inputs_str['post'].to_list()[0:len(input_count_df)]
    final_input_df = final_input_df.append(input_count_df)
    #print(f'Counting done for: {n}')
input_count_str_df = final_input_df.applymap(str)
input_count_str_df.head()

### 2.3. For all outputs

In [ ]:
# LOOPING. Counting outputs and ouputs per ID, option joining dataframes
final_output_df = pd.DataFrame()
for n in neurons_outputs['pre'].unique():
    outputs_count = {}
    curr_outputs = neurons_outputs[neurons_outputs['pre'] == n]
    outputs_str = curr_outputs.applymap(str)
    
    for c in outputs_str['post'].to_list():
        outputs_count[c] = outputs_count.get(c, 0) + 1
    output_count_df = pd.DataFrame(outputs_count, index=[0])
    output_count_df = output_count_df.T
    output_count_df.rename(columns={0: "counts"},inplace=True)
    output_count_df.index.names = ['postsynaptic_ID']
    output_count_df = output_count_df.sort_values(by="counts",ascending=False)
    output_count_df['presynaptic_ID'] = outputs_str['pre'].to_list()[0:len(output_count_df)]
    final_output_df = final_output_df.append(output_count_df)
    #print(f'Counting done for: {n}')
output_count_str_df = final_output_df.applymap(str)
output_count_str_df.head()

### 3. Adding useful information to the dataframe

In [ ]:
# Selecting dataframe
partner_ID = input_count_str_df.index.tolist()
#partner_ID = output_count_str_df.index.tolist()

In [ ]:
#Updating the IDs via Fafbseg

updated_ID_df = fafbseg.flywire.update_ids(partner_ID, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)

partner_ID_ls = updated_ID_df["new_id"].tolist()

In [ ]:
# Identifying user-based annotations about cell identity

identification_df = flywire.find_celltypes(partner_ID_ls, user=None, exact=False, case=False, regex=True, update_roots=False)
identification_no_duplicates_df = identification_df.drop_duplicates(subset='pt_root_id', keep='last', inplace=False, ignore_index=False).copy()

In [ ]:
identification_no_duplicates_df

In [ ]:
# Adding info to the current data set. The function:

def update_dataframe_single_column(source_df, target_df, reference_column):
    # Create a dictionary mapping from the reference column to the source DataFrame
    reference_dict = source_df.groupby(reference_column).first().reset_index().to_dict(orient='records')
    reference_dict = {row[reference_column]: row for row in reference_dict}

    # Update the target DataFrame based on the reference column
    for i, row in target_df.iterrows():
        ref = row[reference_column]
        if ref in reference_dict:
            source_row = reference_dict[ref]
            target_df.loc[i] = source_row

    return target_df


In [ ]:
# Run only for inputs


# Selecting dataframes and resetting index
source_df = identification_no_duplicates_df.copy()
source_df.reset_index(inplace = True, drop = True)
target_df = input_count_str_df.copy()
target_df.reset_index(inplace = True, drop = True)


# Adding columns for the function to properly work
target_df['partner_ID'] = input_count_str_df.index.astype(str)
source_df['partner_ID'] = identification_no_duplicates_df['pt_root_id'].tolist()
target_df['name'] = None
source_df['name'] = identification_no_duplicates_df['tag'].tolist()
target_df['author'] = None
source_df['author'] = identification_no_duplicates_df['user_id'].tolist()

# Function inputs
source_cols = ['name', 'author','partner_ID']
target_cols = ['name', 'author', 'partner_ID']
reference_column = 'partner_ID'

source_df = source_df[source_cols].copy()
target_df = target_df[source_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)


# Running the function and compleating the dataset
result_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_df['counts'] = input_count_str_df['counts'].tolist()
result_df['postsynaptic_ID'] = input_count_str_df['postsynaptic_ID'].tolist()
result_df.head()




In [ ]:
# Run only for outputs


# Selecting dataframes and resetting index
source_df = identification_no_duplicates_df.copy()
source_df.reset_index(inplace = True, drop = True)
target_df = output_count_str_df.copy()
target_df.reset_index(inplace = True, drop = True)


# Adding columns for the function to properly work
target_df['partner_ID'] = output_count_str_df.index.astype(str)
source_df['partner_ID'] = identification_no_duplicates_df['pt_root_id'].tolist()
target_df['name'] = None
source_df['name'] = identification_no_duplicates_df['tag'].tolist()
target_df['author'] = None
source_df['author'] = identification_no_duplicates_df['user_id'].tolist()

# Function inputs
source_cols = ['name', 'author','partner_ID']
target_cols = ['name', 'author', 'partner_ID']
reference_column = 'partner_ID'

source_df = source_df[source_cols].copy()
target_df = target_df[source_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)


# Running the function and compleating the dataset
result_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_df['counts'] = output_count_str_df['counts'].tolist()
result_df['presynaptic_ID'] = output_count_str_df['presynaptic_ID'].tolist()
result_df.head()




In [ ]:
# Saving data in your computer
outDir = r'E:\Connectomics-Data\FlyWire\Excels\min-score-50'
save_excel_file = True

if save_excel_file: 
    ## Input count
    #file_name = f'Tm16_neurons_output_count_R.xlsx'
    file_name = f'Tm1_neurons_input_count_L_remaining_4.xlsx'
    savePath = os.path.join(outDir, file_name)
    result_df.to_excel(savePath, sheet_name='Buhmann synapses')

In [ ]:
#Saving more data in the same file
from openpyxl import load_workbook
updated_ID_df_str =  updated_ID_df.applymap(str)
filePath = savePath

book = load_workbook(filePath)
writer = pd.ExcelWriter(filePath, engine = 'openpyxl')
writer.book = book

updated_ID_df_str.to_excel(writer, sheet_name='ID_update')
writer.save()
writer.close()

## C) Proofreading predicted synapses via annotations


In [ ]:
# Manually loading the URL containing synapses as annotations and notes in each

URL = 'https://ngl.flywire.ai/?json_url=https://global.daf-apis.com/nglstate/api/v1/5833034053976064'
dict_URL = fafbseg.flywire.decode_url(URL)


In [ ]:
# For debuggin purposes

#URL = 'https://ngl.flywire.ai/?json_url=https://global.daf-apis.com/nglstate/api/v1/5574105810075648'# OLD URL
#URL = 'https://ngl.flywire.ai/?json_url=https://global.daf-apis.com/nglstate/api/v1/6667315345096704'# UPDATED URL
URL = 'https://ngl.flywire.ai/?json_url=https://global.daf-apis.com/nglstate/api/v1/6516888544739328' # No segments in the dict
dict_URL = fafbseg.flywire.decode_url(URL)

### 1. Loading URLs from excell file


In [ ]:
# Loading the URLs from an excel file

# Choose path and file
dataPath = r'C:\Users\sebas\Downloads'
fileName = 'All_Tm9_neurons_input_count_ME_R_20221017.xlsx'
filePath = os.path.join(dataPath,fileName)

df = pd.read_excel(filePath)

URL_ls = df['URL buhmann postsynapses']

#URL_ls =['https://ngl.flywire.ai/?json_url=https://global.daf-apis.com/nglstate/api/v1/5833034053976064','https://ngl.flywire.ai/?json_url=https://global.daf-apis.com/nglstate/api/v1/6635071146557440']

In [ ]:
URL_ls

In [ ]:
URL_ls = URL_ls[URL_ls.notnull()] # Getting rid of NaNs 

### 2. Extracting annotations' labels / comments

In [ ]:
# Extracting points and labels/descriptions
points = []
descriptions = []
segment_ids = []
faulty_URLs = []

for URL in URL_ls:
    dict_URL = fafbseg.flywire.decode_url(URL)
    for ann in dict_URL['annotations']:
        if 'point' in ann.keys():
            try: 
                segment_ids.append(ann['segments'])
            except:# dealing with faulty URLs with no data inside
                faulty_URLs.append(URL)
                continue
            try:
                descriptions.append(ann['description'])
            except: # dealing with faulty URLs with no data inside
                descriptions.append('NO DESCRIPTION YET')
        
            points.append(ann['point'])
            
# Splitting in pre and post IDs
pre = []
post = []
for pair in segment_ids:
    pre.append(pair[0])
    post.append(pair[1])   
    
#  len([d for d in descriptions if d.lower() == 'true' or d.lower() == 'true\n' ]) # Brief count of true synapses

In [ ]:
# Creating the summary data frame
proofread_synapses_df = pd.DataFrame()

proofread_synapses_df['post_pt_position'] = points
proofread_synapses_df['proofread_label'] = descriptions
proofread_synapses_df['pre_id'] = pre
proofread_synapses_df['post_id'] = post
    

In [ ]:
# Check if there was any faulta URL
faulty_URLs

In [ ]:
#Visualize your current annotation table
proofread_synapses_df[proofread_synapses_df['proofread_label']!= 'NO DESCRIPTION YET'].head()

### 3. Quantifiying and showing annotation proofread status

In [ ]:
# Current total number of synapses proofread as TRUE from all links in the file:
len([d for d in descriptions if d.lower() == 'true' or d.lower() == 'true\n' ])

In [ ]:
# Current total number of synapses proofread as FALSE from all links in the file:
len([d for d in descriptions if d.lower() == 'false' or d.lower() == 'false\n' ])

In [ ]:
# Current total number of synapses proofread as FALSE NEGATIVE from all links in the file:
len([d for d in descriptions if d.lower() == 'false negative' or d.lower() == 'false negative\n' ])

### 4. Comparing annotation points from 2 URLs

In [ ]:
# Compare number as well as XYZ location

## D) Calculating and plotting synaptic cleft size ditribution between partners 

In [ ]:
# TO BE DONE
# 1. Load Tm9 ID's from excel file. 
# 2. Get main input (probably L3)
# 3. Get individual synapses (pre and post sites)
# 4. Calculate the distance between the two points
# 5. Plot the distance ditribition for a given Tm9-L3 pair
# 6. Plot the median of all Tm9-L3 distribution as a distribution

## E) Checking the existance of IDs from Codex in our data base 

In [ ]:
#Loading Codex file

# Choose path and file
dataPath = r'C:\Users\sebas\Downloads'
fileName = 'root_ids_Tm9_447.txt'
filePath = os.path.join(dataPath,fileName)

#Read txt file
from numpy import loadtxt
codex_ID_ls = list(loadtxt(filePath, comments="#", delimiter=",", unpack=False, dtype='str'))


In [ ]:
#Loading your IDs

# Choose path and file
dataPath = r'C:\Users\sebas\Downloads'
fileName = 'Tm9 proofreadings.xlsx'
filePath = os.path.join(dataPath,fileName)
df = pd.read_excel(filePath)
your_ID_ls = df["seg_id"].astype(str).tolist()



In [ ]:
# Updatind IDs using FAFBSEG
new_codex_ID_df = flywire.update_ids(codex_ID_ls, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
new_codex_ID_ls = new_codex_ID_df["new_id"].tolist()

new_your_ID_df = flywire.update_ids(your_ID_ls, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
new_your_ID_ls = new_your_ID_df["new_id"].tolist()

In [ ]:
#Checking the new IDs in codex ID list with respect to your ID list
print(f"Expected lenght for ID_diference: {len(new_codex_ID_ls)-len(new_your_ID_ls)}")
ID_diference = set(new_codex_ID_ls) - set(new_your_ID_ls)
ID_diference_ls = list(ID_diference)
print(f"Current difference: {len(ID_diference_ls)}")

In [ ]:
#Checking in URL
old_url = flywire.encode_url(your_ID_ls)
print(f'CURRENT old segment IDs: {old_url}')
new_url = flywire.encode_url(new_your_ID_ls)
print(f'CURRENT new segment IDs: {new_url}')
new_url = flywire.encode_url(ID_diference_ls)
print(f'ADD segments: {new_url}')


In [ ]:
# Currently, CAVE update not needed:

In [ ]:
# # Updating  chunkedgraph module of CAVE
# codex_ID_ls_int = list(map(int,codex_ID_ls))
# new_codex_ID_ls_int = list(map(client.chunkedgraph.get_latest_roots,codex_ID_ls_int))

# your_ID_ls_int = list(map(int,your_ID_ls))
# new_your_ID_ls_int = list(map(client.chunkedgraph.get_latest_roots,your_ID_ls_int))


In [ ]:
# #Checking the new IDs in codex ID list with respect to your ID list
# print(f"Expected lenght for ID_diference: {len(new_codex_ID_ls_int)-len(new_your_ID_ls_int)}")

In [ ]:
# # Putting the new IDs into a list and getting a new neuroglancer URL
# new_your_ID_ls_arr = np.concatenate(new_your_ID_ls_int, axis=0 )
# new_your_IDs_ls = new_your_ID_ls_arr.tolist()
# old_url = flywire.encode_url(your_IDs_ls)
# print(f'OLD segments: {old_url}')
# new_url = flywire.encode_url(new_your_IDs_ls)
# print(f'NEW segments: {new_url}')

In [ ]:
# # Putting the new IDs into a list and getting a new neuroglancer URL
# new_codex_ID_ls_arr = np.concatenate(new_codex_ID_ls_int, axis=0 )
# new_codex_IDs_ls = new_codex_ID_ls_arr.tolist()
# old_url = flywire.encode_url(codex_IDs_ls)
# print(f'OLD segments: {old_url}')
# new_url = flywire.encode_url(new_codex_IDs_ls)
# print(f'NEW segments: {new_url}')

## F) Updating your IDs

In [ ]:
#Loading your IDs

# Choose path and file
dataPath = r'C:\Users\sebas\Downloads'
fileName = 'Tm9 proofreadings.xlsx'
filePath = os.path.join(dataPath,fileName)
df = pd.read_excel(filePath)
your_ID_ls = df["seg_id"].astype(str).tolist()

#Updating the IDs
new_your_ID_df = flywire.update_ids(your_ID_ls, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)


In [ ]:
# Saving data in your computer
outDir = r'C:\Users\sebas\Downloads'
save_excel_file = True
new_your_ID_df = new_your_ID_df.astype(str)

if save_excel_file: 
    ## Input count
    file_name = f'Update_IDs.xlsx'
    savePath = os.path.join(outDir, file_name)
    new_your_ID_df.to_excel(savePath)

## G) Plotting our Tm9 patches /clusters or neuron types

In [ ]:
#Loading your IDs

# Choose path and file
dataPath = r'C:\Users\sebas\Downloads'
fileName = 'Tm9 proofreadings_20221212.xlsx' # For Tm9 pacthes
#fileName = 'All_Tm9_neurons_input_count_ME_R_20230110.xlsx' # For neuron types
filePath = os.path.join(dataPath,fileName)
df = pd.read_excel(filePath)


### 0. Plotting all twig-proofread neurons

### 1. Plotting all clusters

In [ ]:
# Filter by cluster ID  and generate a Neuroglancer URL

cluster_ids = df["cluster_id"].unique() # For Tm9 patches

for c_id in cluster_ids:
    temp_df = df[df["cluster_id"] == c_id].copy()
    print(f'Number of IDs in cluster {c_id}: {len(temp_df)}')
    temp_url = flywire.encode_url(temp_df["seg_id"].tolist())
    print(f'Cluster {c_id} URL: {temp_url}')
    

### 1. Or, plotting all neuron types

In [ ]:
#Deletting spaces from symbol strings
symbol_no_Spaces = df.symbol.str.strip()
df["symbol"] = symbol_no_Spaces
guess_no_Spaces = df.guess.str.strip()
df["guess"] = guess_no_Spaces

In [ ]:
symbol_ids = df["symbol"].unique() # For identified inputs
guess_ids = df["guess"].unique() # For identified inputs

In [ ]:
guess_ids

In [ ]:
# Generating URLs for each neuron type
for s_id in symbol_ids:
    temp_df = df[df["symbol"] == s_id].copy()
    print(f'Repetitions in the data set for {s_id}: {len(temp_df)}')
    temp_url = flywire.encode_url(temp_df["presynaptic_ID"].tolist())
    print(f'Symbol {s_id} URL: {temp_url}')

In [ ]:
# Generating URLs for each neuron type
for g_id in guess_ids:
    temp_df = df[df["guess"] == g_id].copy()
    print(f'Repetitions in the data set for {g_id}: {len(temp_df)}')
    temp_url = flywire.encode_url(temp_df["presynaptic_ID"].tolist())
    print(f'Guess {g_id} URL: {temp_url}')

## H) Tm9-patch mining

### 1.1 Visualizing neuron of interest to mine

In [ ]:
# Load neuron from data base
reference_neuron_type = 'Tm16'
import os

dataPath = r'D:\Connectomics data\FlyWire\Excels'
fileDate = '20221216'
fileName = f'All_Tm9_neurons_input_count_ME_R_{fileDate}.xlsx'
filePath = os.path.join(dataPath,fileName)
data_base_df = pd.read_excel(filePath)

In [ ]:
#Filtering the dataframe with  the "reference_neuron_type"
reference_neuron_type_df = data_base_df[data_base_df['symbol'] == reference_neuron_type].copy()
# Creting a Neuroglancer view
url = flywire.encode_url(reference_neuron_type_df["presynaptic_ID"].tolist())
print(f'Neurons URL: {url}')

In [ ]:
reference_neuron_type_df.head()

### 1.2 Loading data set

In [ ]:
# Upload all presynaptic inputs for all Tm9 belonging to a patch in a single dataframe
#Loading your IDs

# Choose path and file
import os

dataPath = r'D:\Connectomics data\FlyWire\Excels'
dataPath = r'C:\Users\sebas\Downloads'
fileDate = '20230109'
fileName = f'Tm9 proofreadings_{fileDate}.xlsx'
filePath = os.path.join(dataPath,fileName)
df = pd.read_excel(filePath)


### 2. Cluster selection

In [ ]:
# Selecting cluster based on cluster ID
c_id = 2.0
filtered_df = df[df["cluster_id"] == c_id].copy()
segmentIDs = filtered_df["seg_id"].tolist()


In [ ]:
# Updating the IDs
new_segmentIDs_df = flywire.update_ids(segmentIDs, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
new_segmentIDs = new_segmentIDs_df["new_id"].tolist()


### 3. Finding other Tm9 based on popular presynaptic neurons

In [ ]:
# Building the common presynapses dataframe (no Buhmann cleft_score filter applied)

pre_synapses = flywire.synapses.fetch_synapses(new_segmentIDs, pre=False, post=True, attach=True, 
                                             min_score=0, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")
post_synapses = flywire.synapses.fetch_synapses(new_segmentIDs, pre=True, post=False, attach=True, 
                                             min_score=0, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")

In [ ]:
# Selecting inputs from a single neuropile
neuropile = 'ME_R' # String. 'LO_R', 'ME_R', ...
neuropile_pre_synapses = pre_synapses[pre_synapses['neuropil'] == neuropile].copy()
neuropile_post_synapses = post_synapses[post_synapses['neuropil'] == neuropile].copy()

In [ ]:
# Getting synaptic counts

# Using FAFseg function
synaptic_counts = fafbseg.flywire.synapses.synapse_counts(new_segmentIDs, by_neuropil=False, 
                                        min_score=0,mat='live', batch_size=10, dataset='production')

In [ ]:
# Using my own function
# Getting synapse_count data frame, among other interesting ones
#Combining x,y,z columns for future purposes
combine_xyz(pre_synapses) 
combine_xyz(post_synapses)
count_pre, count_post, synapses_count_updated = synapse_count(pre_synapses, post_synapses)

In [ ]:
print(len(count_pre['presynaptic_ID'].unique()))
index = count_pre.index
print(len(index.unique()))

### 3.1 Getting the most popular presynaptic partners among the neurons in the patch

In [ ]:
#Pair-wise comparisons to find common presynaptic ids and rank them by popularity

count_pre.sort_index(ascending=True)
index_ids = count_pre.index.unique().tolist()
popular_pre_ids_df = pd.DataFrame()
for i in range(len(index_ids)):
    print(f'Comparing: {index_ids[0]}')
    reminding_ids = [s for s in index_ids if s != index_ids[0]]
    #Creating reference df to be compared
    reference_df = count_pre[count_pre.index == index_ids[0]].copy()
    #Pair-wise comparison with other dataframes
    common_df = pd.DataFrame()
    for r_id in reminding_ids:
        compare_to_df = count_pre[count_pre.index == r_id].copy()
        curr_comparison_df = reference_df[reference_df['presynaptic_ID'].isin(compare_to_df['presynaptic_ID'])]
        data_frames = [common_df, curr_comparison_df]
        common_df = pd.concat(data_frames)
        popular_pre_ids = common_df['presynaptic_ID'].value_counts()
        # Putting together all the pair/wise comparisons in a single dataframe
        popular_pre_ids_df = pd.concat([popular_pre_ids_df, popular_pre_ids.to_frame()])
        
    #Removing the id that has done all comparisons
    print(f'Done: {index_ids[0]}')
    index_ids.remove(index_ids[0])
        
        


In [ ]:
# Popularity rank
popular_pre_ids_df.index.value_counts().to_frame(name ='popularity rank')

In [ ]:
# Choosing some postsynaptic partners from the most popular Tm9 presynaptic parters to uncover other Tm9s
popular_pre_ids = [720575940630334720,720575940631727146,720575940626719065,720575940628005007,720575940623001599,720575940614448746]


### 3.2 Connectivity of the popular presynaptic partners

In [ ]:
# Average connectivity with those Tm9s
popular_id_count_ls = []
for p_id in popular_pre_ids:
    
    #Retrieve all postsynaptic partners with min cleft_score 50!:
    post_synapses = flywire.synapses.fetch_synapses(p_id, pre=True, post=False, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")
    post_count_ls = []
    for post_id in new_segmentIDs:
        temp_post_synapses = post_synapses[post_synapses['post']==post_id].copy()
        temp_count = len(temp_post_synapses)
        post_count_ls.append(temp_count)
    popular_id_count_ls.append(post_count_ls)
    
    


In [ ]:
popular_id_count_ls 

In [ ]:
# For specific popular presynaptic partner, based on counts, retrieve postsynaptic parters where we might find other Tm9s
p_id = [720575940614448746]

# Synaptic counts
pre_synapses = flywire.synapses.fetch_synapses(p_id, pre=False, post=True, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")
post_synapses = flywire.synapses.fetch_synapses(p_id, pre=True, post=False, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")

In [ ]:
#Neuropile filter
# Selecting inputs from a single neuropile
neuropile = 'ME_R' # String. 'LO_R', 'ME_R', ...
neuropile_pre_synapses = pre_synapses[pre_synapses['neuropil'] == neuropile].copy()
neuropile_post_synapses = post_synapses[post_synapses['neuropil'] == neuropile].copy()

In [ ]:
# Using FAFseg function
synaptic_counts = fafbseg.flywire.synapses.synapse_counts(p_id, by_neuropil=True, 
                                        min_score=50,mat='live', batch_size=10, dataset='production')


In [ ]:
synaptic_counts

In [ ]:
# Using my own function
# Getting synapse_count data frame, among other interesting ones
#Combining x,y,z columns for future purposes
combine_xyz(neuropile_pre_synapses) 
combine_xyz(neuropile_post_synapses)
count_pre, count_post, synapses_count_updated = synapse_count(neuropile_pre_synapses, neuropile_post_synapses)

In [ ]:
#Getting postsynaptic dis by filtering connectivity
count_post['counts'] = count_post['counts'].astype('int')
post_id_ls = count_post[(count_post['counts']>=5) & (count_post['counts']<=8)]['postsynaptic_ID'].tolist()

#Generate neuroglancer link to visually rvaluate the presence of other Tm9s
url = flywire.encode_url(post_id_ls)
print(f'Postynaptic segments: {url}')


In [ ]:

#Existing
Tm9_ids = [720575940624502013,
720575940617720413,
720575940628205800,
720575940631746090,
720575940633161005,
720575940622801366,
720575940619775281,
720575940612306650,
720575940622355226]

new_Tm9_ids_df = flywire.update_ids(Tm9_ids, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)


In [ ]:
flywire.encode_url(Tm9_ids)

In [ ]:
new_Tm9_ids_df